**Task 1**

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import os
import glob
import re
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset

Check Cuda Availability

In [ ]:
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Mount google drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Define data path

In [ ]:
data_path = '/content/drive/My Drive/xyz/apex'
images = [image for image in os.listdir(data_path) if image.endswith('.jpg')]

In [ ]:
print(images)

['95_88_827bb6d2-110d-11ef-a573-a46bb606fa86.jpg', '121_115_ac9065d6-110c-11ef-a573-a46bb606fa86.jpg', '116_135_85a928b2-110d-11ef-a573-a46bb606fa86.jpg', '96_160_5ebb2d3a-110e-11ef-a573-a46bb606fa86.jpg', '66_111_618aea56-110d-11ef-a573-a46bb606fa86.jpg', '88_125_6245a11a-110e-11ef-a573-a46bb606fa86.jpg', '111_125_beac37fe-110c-11ef-a573-a46bb606fa86.jpg', '125_87_522e169c-1107-11ef-be05-a46bb606fa86.jpg', '58_187_127f2bb2-1111-11ef-a573-a46bb606fa86.jpg', '29_107_48195fa4-110c-11ef-a573-a46bb606fa86.jpg', '125_123_bdf0f340-110c-11ef-a573-a46bb606fa86.jpg', '112_122_f990740a-110e-11ef-a573-a46bb606fa86.jpg', '129_126_57a108c2-110d-11ef-a573-a46bb606fa86.jpg', '100_91_fc648b14-1107-11ef-9bc2-a46bb606fa86.jpg', '108_135_ff253e16-110c-11ef-a573-a46bb606fa86.jpg', '96_138_dd638f9c-110e-11ef-a573-a46bb606fa86.jpg', '86_106_de817062-1107-11ef-9bc2-a46bb606fa86.jpg', '48_110_6690d294-110e-11ef-a573-a46bb606fa86.jpg', '87_79_27710856-1107-11ef-be05-a46bb606fa86.jpg', '85_135_0d4cb410-110d-11e

Feature Extraction

In [ ]:
def extract_coordinates(file_name):
  match = re.match(r'(\d+)_(\d+)_.*\.jpg', file_name)
  if match:
    return list(map(int, match.groups()))
  else:
    return None

In [ ]:
data = []

for image in images:
  coordination = extract_coordinates(image)
  if coordination:
    data.append((os.path.join(data_path, image), coordination))

In [ ]:
print(data)

[('/content/drive/My Drive/xyz/apex/95_88_827bb6d2-110d-11ef-a573-a46bb606fa86.jpg', [95, 88]), ('/content/drive/My Drive/xyz/apex/121_115_ac9065d6-110c-11ef-a573-a46bb606fa86.jpg', [121, 115]), ('/content/drive/My Drive/xyz/apex/116_135_85a928b2-110d-11ef-a573-a46bb606fa86.jpg', [116, 135]), ('/content/drive/My Drive/xyz/apex/96_160_5ebb2d3a-110e-11ef-a573-a46bb606fa86.jpg', [96, 160]), ('/content/drive/My Drive/xyz/apex/66_111_618aea56-110d-11ef-a573-a46bb606fa86.jpg', [66, 111]), ('/content/drive/My Drive/xyz/apex/88_125_6245a11a-110e-11ef-a573-a46bb606fa86.jpg', [88, 125]), ('/content/drive/My Drive/xyz/apex/111_125_beac37fe-110c-11ef-a573-a46bb606fa86.jpg', [111, 125]), ('/content/drive/My Drive/xyz/apex/125_87_522e169c-1107-11ef-be05-a46bb606fa86.jpg', [125, 87]), ('/content/drive/My Drive/xyz/apex/58_187_127f2bb2-1111-11ef-a573-a46bb606fa86.jpg', [58, 187]), ('/content/drive/My Drive/xyz/apex/29_107_48195fa4-110c-11ef-a573-a46bb606fa86.jpg', [29, 107]), ('/content/drive/My Drive

Load Data to x and y (features and label)

In [ ]:
X_array = []
y_array = []

for item in data:
  x_image = cv2.imread(item[0])
  x_image = cv2.resize(x_image, (224, 224))
  X_array.append(x_image)
  y_image = item[1]
  y_array.append(y_image)

X_array = np.array(X_array)
y_array = np.array(y_array)

In [ ]:
np.save('x_coordinates.npy', X_array)
np.save('y_coordinates.npy', y_array)

Convert Data to Tensor

In [ ]:
X = torch.tensor(X_array, dtype=torch.float32)
y = torch.tensor(y_array, dtype=torch.float32)

Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=122)
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

Load ResNet18

In [ ]:
baseModel = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 154MB/s]


Create Model

In [ ]:
hidden_units = 128
output_units = y_train.shape[1]
activation_function = nn.ReLU()

In [ ]:
headModel = nn.AdaptiveAvgPool2d((1,1))
hidden_layer = nn.Linear(1000, hidden_units)
dropout_layer = nn.Dropout(p=0.5)
output_layer = nn.Linear(hidden_units, output_units)
model_resnet18 = nn.Sequential(baseModel, hidden_layer, activation_function, output_layer)
model_resnet18.to(device)

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

Train Data

In [ ]:
epochs = 55
batch_size = 16
loss_function = nn.MSELoss()
num_batches_train = (len(X_train) + batch_size - 1) // batch_size
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)

for epoch in range(epochs):
  loss_value = 0

  for i in range(num_batches_train):
    batch_inputs = X_train[i * batch_size:(i + 1) * batch_size].permute(0, 3, 1, 2) # (N, C, H, W, C, N)
    batch_labels = y_train[i * batch_size:(i + 1) * batch_size]

    optimizer.zero_grad()
    outputs = model_resnet18(batch_inputs)
    loss = loss_function(outputs, batch_labels)
    loss.backward()
    optimizer.step()
    loss_value += loss.item()

  epoch_loss = loss_value / num_batches_train
  print(f'epoch {epoch+1} Train Loss: {epoch_loss}')

epoch 1 Train Loss: 1588.6135806848506
epoch 2 Train Loss: 622.0740450113127
epoch 3 Train Loss: 539.4494766386429
epoch 4 Train Loss: 463.56580738029857
epoch 5 Train Loss: 392.4240538342164
epoch 6 Train Loss: 328.75903516712754
epoch 7 Train Loss: 277.8086883998153
epoch 8 Train Loss: 249.34401989927386
epoch 9 Train Loss: 209.1429525696405
epoch 10 Train Loss: 190.49129826007504
epoch 11 Train Loss: 173.25776634594
epoch 12 Train Loss: 160.81689966787206
epoch 13 Train Loss: 145.70733393300878
epoch 14 Train Loss: 126.47861662005434
epoch 15 Train Loss: 109.11535803162225
epoch 16 Train Loss: 99.79916729313312
epoch 17 Train Loss: 92.00565679946749
epoch 18 Train Loss: 85.89875185371625
epoch 19 Train Loss: 86.01695410095819
epoch 20 Train Loss: 107.91663137756952
epoch 21 Train Loss: 127.8296609255347
epoch 22 Train Loss: 155.1112238817876
epoch 23 Train Loss: 168.93129125916133
epoch 24 Train Loss: 222.10134298494546
epoch 25 Train Loss: 183.7962137165636
epoch 26 Train Loss: 122

Save Model

In [ ]:
torch.save(model_resnet18, 'model_resnet18.pth')

Load Model

In [ ]:
model_resnet18 = torch.load('model_resnet18.pth')

Model Evaluation

In [ ]:
model_resnet18.eval().to(device)

num_batches_test = (len(X_test) + batch_size - 1) // batch_size
actuals = []
predictions = []

with torch.no_grad():
  for i in range(num_batches_test):
    batch_inputs_test = X_test[i * batch_size:(i + 1) * batch_size].permute(0, 3, 1, 2)
    batch_labels_test = y_test[i * batch_size:(i + 1) * batch_size]

    outputs = model_resnet18(batch_inputs_test)
    actuals.append(batch_labels_test.cpu().numpy())
    predictions.append(outputs.cpu().numpy())

actuals = np.concatenate(actuals, axis=0)
predictions = np.concatenate(predictions, axis=0)
print('Mean Squared Error: ', mean_squared_error(actuals, predictions))

Mean Squared Error:  380.43414
